In [ ]:
!pip install transformers torch
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer



In [ ]:
# Load GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
def generate_text_gpt(prompt, max_length=100):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    attention_mask = torch.ones_like(input_ids)

    output_ids = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=max_length,
        pad_token_id=tokenizer.eos_token_id,  # Set pad token to eos token
        num_return_sequences=1,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.9
    )

    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return output_text


In [ ]:
# Example
user_prompt = "The importance of biodiversity"
print(generate_text_gpt(user_prompt))

The importance of biodiversity and the relationship between it and biodiversity can be seen in the fact that in the case of the two species, the human mind and the dog is the most strongly connected. The second species of the world is the chimpanzee and is the most highly connected. By analogy, when one wants to understand an organism, one wants to understand its species and thus to be able to perceive it in all its forms. This is why a human mind is one part of the organism, whereas a


In [ ]:
!pip uninstall tensorflow
!pip install tensorflow-cpu


In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))

TensorFlow version: 2.19.0
GPU Available: []


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [ ]:
corpus = [
    "The quick brown fox jumped over the lazy dog",
    "I am learning how to train a neural network",
    "Deep learning is a subset of machine learning",
    "Neural networks can be trained for various tasks",
    "This is a simple corpus for training the LSTM model"
]

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1


In [ ]:
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)


In [ ]:
max_seq_len = max([len(x) for x in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_seq_len, padding='pre')

In [ ]:
xs, labels = input_sequences[:, :-1], input_sequences[:, -1]
ys = to_categorical(labels, num_classes=total_words)

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # This disables the GPU


In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(10,)),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')

# Create some dummy data
import numpy as np
x = np.random.random((100, 10))
y = np.random.random((100, 1))

model.fit(x, y, epochs=5)



Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.4862
Epoch 2/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.3682 
Epoch 3/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.2732
Epoch 4/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.1636
Epoch 5/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.1294


In [ ]:
def generate_text_lstm(seed_text, next_words=20):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_seq_len-1, padding='pre')
        predicted = model.predict(token_list, verbose=0).argmax(axis=-1)[0]
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

In [ ]:
def preprocess_input(text, tokenizer, max_seq_len):
    # Tokenize the input text
    token_list = tokenizer.texts_to_sequences([text])[0]

    # Pad sequences to make sure they are the right length
    token_list = pad_sequences([token_list], maxlen=max_seq_len, padding='pre')  # Use max_seq_len, not max_seq_len-1
    return token_list


AttributeError: 'list' object has no attribute 'shape'

In [ ]:
import numpy as np

def predict_next_word(model, token_list):
    # Predict the next word
    predicted = model.predict(token_list, verbose=0)
    predicted_word_index = np.argmax(predicted, axis=-1)[0]
    return predicted_word_index


In [ ]:
def get_predicted_word(predicted_word_index, tokenizer):
    # Reverse the tokenizer index
    for word, index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word
    return None


In [ ]:
def generate_text_lstm(seed_text, model, tokenizer, max_seq_len, next_words=20):
    for _ in range(next_words):
        # Preprocess the seed text
        token_list = preprocess_input(seed_text, tokenizer, max_seq_len)

        # Predict the next word
        predicted_word_index = predict_next_word(model, token_list)

        # Get the predicted word
        predicted_word = get_predicted_word(predicted_word_index, tokenizer)

        # Append the word to the seed text
        seed_text += " " + predicted_word

    return seed_text


In [ ]:
# Assuming `model`, `tokenizer`, and `max_seq_len` are already defined (after training the model)
seed_text = "The importance of biodiversity"
generated_text = generate_text_lstm(seed_text, model, tokenizer, max_seq_len, next_words=50)
print(generated_text)


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_1" is incompatible with the layer: expected axis -1 of input shape to have value 10, but received input with shape (1, 9)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(1, 9), dtype=int32)
  • training=False
  • mask=None